In [3]:
import pandas as pd

In [1]:
from model_utils.utils_s2 import Predictor

In [2]:
predictor = Predictor('result/synthetic_s2_training_2/model_ckpt_ep_12.pth')

In [4]:
# toy example?


'''
   bb_x  bb_y  siz_x  siz_y bb_type  n_proposal  prob_median  n_proposal_norm
0     4     9      6      6   hloop          15     0.930053         0.416667
1     1    12      4      4    stem          16     0.925148         1.000000
2    41    50     10     10   hloop          45     0.992122         0.450000
3    38    53      4      4    stem          16     0.994010         1.000000
4    36    55      3      3   iloop           9     0.995822         1.000000
5    31    60      6      6    stem          36     0.978096         1.000000
'''


stems = pd.DataFrame([{'bb_x': 1, 'bb_y': 12, 'siz_x': 4, 'siz_y': 4, 'prob_median': 0.93, 'n_proposal_norm': 1.0},
                     {'bb_x': 38, 'bb_y': 53, 'siz_x': 4, 'siz_y': 4, 'prob_median': 0.99, 'n_proposal_norm': 1.0},
                     {'bb_x': 31, 'bb_y': 60, 'siz_x': 6, 'siz_y': 6, 'prob_median': 0.98, 'n_proposal_norm': 1.0}])

iloops = pd.DataFrame([{'bb_x': 36, 'bb_y': 55, 'siz_x': 3, 'siz_y': 3, 'prob_median': 0.99, 'n_proposal_norm': 1.0}])

hloops = pd.DataFrame([{'bb_x': 4, 'bb_y': 9, 'siz_x': 6, 'siz_y': 6, 'prob_median': 0.93, 'n_proposal_norm': 0.83},
                      {'bb_x': 41, 'bb_y': 50, 'siz_x': 10, 'siz_y': 10, 'prob_median': 0.99, 'n_proposal_norm': 0.9}])



In [5]:
predictor.predict(stems, iloops, hloops)

NameError: name 'self' is not defined